In [3]:
import os, time
import math
import numpy as np
from torchvision import datasets, transforms
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import imageio

In [2]:
epochs = 25
batch_size = 64
latent_dim = 100
img_size = 32

In [3]:
cuda = True if torch.cuda.is_available() else False

In [4]:
if not os.path.isdir('sample_images'):
    os.mkdir('sample_images')

In [5]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [6]:
class Generator(nn.Module):
    # initializers
    def __init__(self, d=16):
        super(Generator, self).__init__()
        self.init_size = img_size // 16
        self.rel = nn.LeakyReLU(0.2, inplace=True)
        self.l1 = nn.Sequential(nn.Linear(100, 128 * self.init_size ** 2))
        self.deconv1 = nn.ConvTranspose2d(128, 64, 4, 2, 1)
        self.deconv1_bn = nn.BatchNorm2d(64)
        self.deconv2 = nn.ConvTranspose2d(64, 32, 4, 2, 1)
        self.deconv2_bn = nn.BatchNorm2d(32)
        self.deconv3 = nn.ConvTranspose2d(32, 16, 4, 2, 1)
        self.deconv3_bn = nn.BatchNorm2d(16)
        self.deconv4 = nn.ConvTranspose2d(16, 1, 4, 2, 1)

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, input):
        # x = F.relu(self.deconv1(input))
        x = self.l1(input)
        x = x.view(x.shape[0], 128, self.init_size, self.init_size)
        x = self.rel(self.deconv1_bn(self.deconv1(x)))
        x = self.rel(self.deconv2_bn(self.deconv2(x)))
        x = self.rel(self.deconv3_bn(self.deconv3(x)))
        x = F.tanh(self.deconv4(x))

        return x

In [7]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.convs = nn.Sequential(
            *discriminator_block(1, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        ds_size = img_size // 2 ** 4
        self.fc_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())

    def forward(self, img):
        x = self.convs(img)
        x = out.view(x.shape[0], -1)
        x = self.fc_layer(x)

        return x

In [8]:
adversarial_loss = torch.nn.BCELoss()

generator = Generator()
discriminator = Discriminator()

In [9]:
generator

Generator(
  (rel): LeakyReLU(negative_slope=0.2, inplace=True)
  (l1): Sequential(
    (0): Linear(in_features=100, out_features=512, bias=True)
  )
  (deconv1): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (deconv1_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (deconv2): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (deconv2_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (deconv3): ConvTranspose2d(32, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (deconv3_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (deconv4): ConvTranspose2d(16, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
)

In [10]:
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [11]:
# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Discriminator(
  (convs): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout2d(p=0.25, inplace=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Dropout2d(p=0.25, inplace=False)
    (6): BatchNorm2d(32, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Dropout2d(p=0.25, inplace=False)
    (10): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (12): LeakyReLU(negative_slope=0.2, inplace=True)
    (13): Dropout2d(p=0.25, inplace=False)
    (14): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc_layer

In [12]:
transform = transforms.Compose([
        transforms.Scale(img_size),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
])

/home/sauban/.local/lib/python3.6/site-packages/torchvision/transforms/transforms.py:220: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [13]:
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST('data2', train=True, download=True, transform=transform),
    batch_size=batch_size, shuffle=True)

In [14]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [15]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [16]:
print('training start!')
start_time = time.time()
for epoch in range(epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        #  Train Generator
        
        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)
        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        
        optimizer_D.zero_grad()

        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()
        
        if i % 100 == 0:
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, epochs, i, len(dataloader), d_loss.item(), g_loss.item())
            )

        batches_done = epoch * len(dataloader) + i
    save_image(gen_imgs.data[:25], "sample_images/%d.png" % epoch, nrow=5, normalize=True)
end_time = time.time()
total_time = end_time - start_time
total_time

/home/sauban/.local/lib/python3.6/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


training start!
[Epoch 0/25] [Batch 0/938] [D loss: 0.693135] [G loss: 0.688521]
[Epoch 0/25] [Batch 100/938] [D loss: 0.078718] [G loss: 2.480105]
[Epoch 0/25] [Batch 200/938] [D loss: 0.274360] [G loss: 2.432323]
[Epoch 0/25] [Batch 300/938] [D loss: 0.563196] [G loss: 1.030362]
[Epoch 0/25] [Batch 400/938] [D loss: 0.443058] [G loss: 1.651758]
[Epoch 0/25] [Batch 500/938] [D loss: 0.437042] [G loss: 1.696185]
[Epoch 0/25] [Batch 600/938] [D loss: 0.725458] [G loss: 1.040406]
[Epoch 0/25] [Batch 700/938] [D loss: 0.493642] [G loss: 1.081107]
[Epoch 0/25] [Batch 800/938] [D loss: 0.392997] [G loss: 1.797325]
[Epoch 0/25] [Batch 900/938] [D loss: 0.452959] [G loss: 0.989500]
[Epoch 1/25] [Batch 0/938] [D loss: 0.533845] [G loss: 1.087710]
[Epoch 1/25] [Batch 100/938] [D loss: 0.514282] [G loss: 1.125285]
[Epoch 1/25] [Batch 200/938] [D loss: 0.706578] [G loss: 1.006861]
[Epoch 1/25] [Batch 300/938] [D loss: 0.682207] [G loss: 1.055139]
[Epoch 1/25] [Batch 400/938] [D loss: 0.465723] [G

[Epoch 12/25] [Batch 300/938] [D loss: 0.606479] [G loss: 0.766586]
[Epoch 12/25] [Batch 400/938] [D loss: 0.619509] [G loss: 0.827589]
[Epoch 12/25] [Batch 500/938] [D loss: 0.561211] [G loss: 0.745492]
[Epoch 12/25] [Batch 600/938] [D loss: 0.616167] [G loss: 0.823430]
[Epoch 12/25] [Batch 700/938] [D loss: 0.572836] [G loss: 0.661386]
[Epoch 12/25] [Batch 800/938] [D loss: 0.760903] [G loss: 0.795779]
[Epoch 12/25] [Batch 900/938] [D loss: 0.611744] [G loss: 0.673179]
[Epoch 13/25] [Batch 0/938] [D loss: 0.693281] [G loss: 0.638996]
[Epoch 13/25] [Batch 100/938] [D loss: 0.715883] [G loss: 0.802268]
[Epoch 13/25] [Batch 200/938] [D loss: 0.638263] [G loss: 0.751245]
[Epoch 13/25] [Batch 300/938] [D loss: 0.684516] [G loss: 1.104545]
[Epoch 13/25] [Batch 400/938] [D loss: 0.708819] [G loss: 0.627218]
[Epoch 13/25] [Batch 500/938] [D loss: 0.579914] [G loss: 0.789038]
[Epoch 13/25] [Batch 600/938] [D loss: 0.753861] [G loss: 0.617646]
[Epoch 13/25] [Batch 700/938] [D loss: 0.548656] [

[Epoch 24/25] [Batch 400/938] [D loss: 0.468619] [G loss: 0.675721]
[Epoch 24/25] [Batch 500/938] [D loss: 0.329600] [G loss: 0.785363]
[Epoch 24/25] [Batch 600/938] [D loss: 0.261706] [G loss: 1.081682]
[Epoch 24/25] [Batch 700/938] [D loss: 0.274637] [G loss: 1.414595]
[Epoch 24/25] [Batch 800/938] [D loss: 0.472698] [G loss: 1.307553]
[Epoch 24/25] [Batch 900/938] [D loss: 0.247356] [G loss: 1.705978]


569.8033390045166

In [17]:
epochs

25

In [4]:
images = []
for epoch in range(25):
    img_name = 'sample_images/' + str(epoch) + '.png'
    images.append(imageio.imread(img_name))
imageio.mimsave('sample_images/animation.gif', images, fps=3)